<a href="https://colab.research.google.com/github/aditi980/Fakenewsdetector/blob/main/Fakenews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import sklearn
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

In [4]:
data_fake = pd.read_csv('/content/Fake.csv')
data_true = pd.read_csv('/content/True.csv', on_bad_lines='skip')

In [5]:
data_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [6]:
data_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [7]:
data_fake["class"] = 0
data_true["class"] = 1

In [8]:
data_fake.shape , data_true.shape

((23481, 5), (4925, 5))

In [12]:
data_fake_manual_testing = data_fake.tail(10)
for i in range(23480,23470,-1):
  data_fake.drop([i], axis = 0, inplace = True)

data_true_manual_testing = data_true.tail(10)
for i in range(4924,4914,-1):
  data_true.drop([i], axis = 0, inplace = True)

In [13]:
 data_fake.shape , data_true.shape

((23471, 5), (4915, 5))

In [14]:
data_fake_manual_testing['class'] = 0
data_true_manual_testing['class'] = 1

/tmp/ipython-input-277247672.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_fake_manual_testing['class'] = 0
/tmp/ipython-input-277247672.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_true_manual_testing['class'] = 1


In [15]:
data_fake_manual_testing.head()

,title,text,subject,date,class
23471,Seven Iranians freed in the prisoner swap have...,"21st Century Wire says This week, the historic...",Middle-east,"January 20, 2016",0
23472,#Hashtag Hell & The Fake Left,By Dady Chery and Gilbert MercierAll writers ...,Middle-east,"January 19, 2016",0
23473,Astroturfing: Journalist Reveals Brainwashing ...,Vic Bishop Waking TimesOur reality is carefull...,Middle-east,"January 19, 2016",0
23474,The New American Century: An Era of Fraud,Paul Craig RobertsIn the last years of the 20t...,Middle-east,"January 19, 2016",0
23475,Hillary Clinton: ‘Israel First’ (and no peace ...,Robert Fantina CounterpunchAlthough the United...,Middle-east,"January 18, 2016",0


In [16]:
data_true_manual_testing.head()

,title,text,subject,date,class
4915,Trump budget to give first look at infrastruct...,WASHINGTON (Reuters) - President Donald Trump ...,politicsNews,"May 19, 2017",1
4916,Trump faces diplomatic hurdles during 28 hours...,JERUSALEM (Reuters) - U.S. presidential trips ...,politicsNews,"May 19, 2017",1
4917,Trump under pressure to assure NATO allies he ...,WASHINGTON (Reuters) - Standing in front of a ...,politicsNews,"May 19, 2017",1
4918,"Trump denies asking Comey to drop probe, decri...",WASHINGTON (Reuters) - U.S. President Donald T...,politicsNews,"May 18, 2017",1
4919,"For Trump's defenders, White House turmoil is ...","WASHINGTON (Reuters) - In Washington, the chat...",politicsNews,"May 18, 2017",1


In [17]:
data_merge = pd.concat([data_fake, data_true], axis =0 )
data_merge.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [18]:
data_merge.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

In [22]:
data = data_merge.drop(['title','subject','date'], axis = 1)

In [23]:
data.isnull().sum()

,0
text,0
class,0


In [24]:
data = data.sample(frac = 1)

In [25]:
data.head()

,text,class
182,WASHINGTON (Reuters) - Republican leaders in t...,1
9090,This would be a laugh a minute except for the ...,0
18780,OMG. Why do these two gems of social media not...,0
1834,"Let s face it, if it weren t for gerrymanderin...",0
13302,If you ve ever watched CSPAN you ll know how r...,0


In [27]:
data.reset_index(inplace = True)
data.drop(['index'], axis = 1, inplace = True)

In [28]:
data.columns

Index(['text', 'class'], dtype='object')

In [29]:
data.head()

,text,class
0,WASHINGTON (Reuters) - Republican leaders in t...,1
1,This would be a laugh a minute except for the ...,0
2,OMG. Why do these two gems of social media not...,0
3,"Let s face it, if it weren t for gerrymanderin...",0
4,If you ve ever watched CSPAN you ll know how r...,0


In [30]:
def wordopt(text):
  text = text.lower()
  text = re.sub('\[.*?\]', '', text)
  text = re.sub("\\W", " ", text)
  text = re.sub('https?://\S+|www\.\S+', ' ', text)
  text = re.sub('<.*?.+', '', text)
  text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
  text = re.sub('\n', '', text)
  text = re.sub('\w*\d\w*', '', text)
  return text

<>:3: SyntaxWarning: invalid escape sequence '\['
<>:5: SyntaxWarning: invalid escape sequence '\S'
<>:9: SyntaxWarning: invalid escape sequence '\w'
<>:3: SyntaxWarning: invalid escape sequence '\['
<>:5: SyntaxWarning: invalid escape sequence '\S'
<>:9: SyntaxWarning: invalid escape sequence '\w'
/tmp/ipython-input-3620435347.py:3: SyntaxWarning: invalid escape sequence '\['
  text = re.sub('\[.*?\]', '', text)
/tmp/ipython-input-3620435347.py:5: SyntaxWarning: invalid escape sequence '\S'
  text = re.sub('https?://\S+|www\.\S+', ' ', text)
/tmp/ipython-input-3620435347.py:9: SyntaxWarning: invalid escape sequence '\w'
  text = re.sub('\w*\d\w*', '', text)


In [31]:
data['text'] = data['text'].apply(wordopt)

In [32]:
x = data['text']
y = data['class']

In [33]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.25)

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

In [36]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()
LR.fit(xv_train, y_train)

LogisticRegression()

In [37]:
pred_lr = LR.predict(xv_test)

In [38]:
LR.score(xv_test, y_test)

0.9867549668874173

In [39]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      5883
           1       0.98      0.94      0.96      1214

    accuracy                           0.99      7097
   macro avg       0.99      0.97      0.98      7097
weighted avg       0.99      0.99      0.99      7097



In [40]:
from sklearn.tree import DecisionTreeClassifier

DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

In [41]:
pred_dt = DT.predict(xv_test)

In [42]:
DT.score(xv_test, y_test)

0.9954910525574187

In [46]:
print(classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5883
           1       0.99      0.99      0.99      1214

    accuracy                           1.00      7097
   macro avg       0.99      0.99      0.99      7097
weighted avg       1.00      1.00      1.00      7097



In [47]:
from sklearn.ensemble import GradientBoostingClassifier

GBC = GradientBoostingClassifier(random_state = 0)
GBC.fit(xv_train, y_train)

GradientBoostingClassifier(random_state=0)

In [48]:
pred_gbc = GBC.predict(xv_test)

In [49]:
GBC.score(xv_test, y_test)

0.9946456249119346

In [50]:
print(classification_report(y_test, pred_gbc))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      5883
           1       0.97      1.00      0.98      1214

    accuracy                           0.99      7097
   macro avg       0.99      1.00      0.99      7097
weighted avg       0.99      0.99      0.99      7097



In [51]:
from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier(random_state = 0)
RF.fit(xv_train, y_train)

RandomForestClassifier(random_state=0)

In [52]:
pred_RF = RF.predict(xv_test)

In [53]:
GBC.score(xv_test, y_test)

0.9946456249119346

In [54]:
print(classification_report(y_test, pred_RF))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      5883
           1       1.00      0.89      0.94      1214

    accuracy                           0.98      7097
   macro avg       0.99      0.95      0.97      7097
weighted avg       0.98      0.98      0.98      7097



In [59]:
def output_label(n):
  if n == 0:
    return "Fake News"
  elif n == 1:
    return "True News"

def manual_testing(news):
  testing_news = {"text":[news]}
  new_def_test = pd.DataFrame(testing_news)
  new_def_test['text'] = new_def_test['text'].apply(wordopt)
  new_x_test = new_def_test['text']
  news_xv_test = vectorization.transform(new_x_test)
  pred_LR = LR.predict(news_xv_test)
  pred_DT = DT.predict(news_xv_test)
  pred_GBC = GBC.predict(news_xv_test)
  print("\n\nLR Prediction: {} \nDT Prediction: {} \nGBC Prediction: {}".format(output_label(pred_LR[0]), output_label(pred_DT[0]), output_label(pred_GBC[0])))

In [ ]:
news = string(input())
manual_testing(news)